In [ ]:
"""In this assignment, you should train your own net on cifar10 classification with deep learning framework MXNet.
   With MXNet, you only need to define the nets with symbol connection, then set hyperparameters to train the 
   network. You can also save your model and load pretrained model to finetune the network. Make sure using GPU 
   mode. You should achieve at least 80% on the validation set."""

"""vist http://mxnet.io/get_started/index.html to get familar with mxnet!"""
   
import sys
import os
import numpy as np
import mxnet as mx
import logging

# download data if necessary
def _download(data_dir):
    if not os.path.isdir(data_dir):
        os.system("mkdir " + data_dir)
    os.chdir(data_dir)
    if (not os.path.exists('train.rec')) or \
       (not os.path.exists('test.rec')) :
        os.system("wget http://data.dmlc.ml/mxnet/data/cifar10.zip")
        os.system("unzip -u cifar10.zip")
        os.system("mv cifar/* .; rm -rf cifar; rm cifar10.zip")
    os.chdir("..")


# data
def get_iterator(data_shape=(3, 28, 28)):
    if '://' not in data_dir:
        _download(data_dir)

    train = mx.io.ImageRecordIter(
        path_imgrec = os.path.join(data_dir, "train.rec"),
        mean_img    = os.path.join(data_dir, "mean.bin"),
        data_shape  = data_shape,
        batch_size  = batch_size,
        rand_crop   = True,
        rand_mirror = True)

    val = mx.io.ImageRecordIter(
        path_imgrec = os.path.join(data_dir, "test.rec"),
        mean_img    = os.path.join(data_dir, "mean.bin"),
        rand_crop   = False,
        rand_mirror = False,
        data_shape  = data_shape,
        batch_size  = batch_size)

    return (train, val)


def get_net(num_classes=10):
    #####################################################################################
    # TODO: define your net                                                             #
    # Define symbols that using convolution and max pooling to extract better features  #
    # from input image.                                                                 #
    #####################################################################################
    
    data=mx.sym.Varlable(name='data')
    conv1=mx.sym.Convolution(data=data,name='conv1',kernel=(5,5),num_filter=100)
    bn1=mx.sym.BatchNorm(data=conv1,name='bn1')
    relu1=mx.sym.Activation(data=bn1,act_type='relu')
    pool1=mx.sym.Pooling(data=relu1,name='pool1',pool_type='max',kernel=(2,2),stride=(2,2))
    
    conv2=mx.sym.Convolution(data=pool1,name='conv2',kernel=(5,5),num_filter=100)
    bn2=mx.sym.BatchNorm(data=conv2,name='bn1')
    relu2=mx.sym.Activation(data=bn2,act_type='relu')
    pool2=mx.sym.Pooling(data=relu2,name='pool1',pool_type='max',kernel=(2,2),stride=(2,2))
    pool2=mx.sym.Flatten(data=pool2)
    
    fc1=mx.sym.FullyConnected(data=pool2,name='fc1',num_hidden=1024)
    bn3=mx.sym.BatchNorm(data=fc1,name='bn3')
    relu3=mx.sym.Activation(data=bn3,act_type='relu')
    
    fc2=mx.sym.FullyConnected(data=relu3,name='fc2',num_hidden=num_classes)
    softmax=mx.sym.SoftmaxOutput(data=fc2,name='softmax')
    
    #####################################################################################
    #                              END OF YOUR CODE                                     #
    #####################################################################################
    return softmax

In [ ]:
network = get_net()

################################################################################
# TODO: this is similar as solver                                              #
################################################################################

############################ set hyperparameters ###############################
batch_size = 128
weight_decay = 1e-9  # same as weight reg
num_epoch = 100
learning_rate = 1e-2 
devs=mx.gpu(1)     # set device id

################################  path #########################################
data_dir = 'cifar10/'
chk_dir = 'model/'
chk_prefix = chk_dir +'net1'
load_model = False   ## set true if you want to load a pretrained model and finetune with lower learning rate

if not os.path.isdir(chk_dir):
     os.system("mkdir " + chk_dir)

reload(logging)
head = '%(asctime)-15s %(message)s'
logging.basicConfig(level=logging.DEBUG, format=head)

eval_metrics = ['accuracy']

## TopKAccuracy only allows top_k > 1
#eval_metrics.append(mx.metric.create('top_k_accuracy', top_k = 5))

if load_model:
    model_prefix = 'model/net1'
    model_iter = 30  # which model to load

    _, arg_params,__ = mx.model.load_checkpoint(model_prefix, model_iter)
else:
    arg_params = None
    model_iter = 0

model=mx.model.FeedForward(
       ctx      = devs,
       symbol   = network,
       arg_params = arg_params,
       begin_epoch = model_iter,
       num_epoch  = num_epoch,
       learning_rate = learning_rate,
       momentum      = 0.9,
       wd            = weight_decay,
      initializer   = mx.init.Xavier(factor_type='in', magnitude=2.34)    ## weight initialization
       )

train_ite, val_ite = get_iterator()
model.fit(
        X          = train_ite,
        eval_data  = val_ite,
        eval_metric = eval_metrics,
        batch_end_callback = mx.callback.Speedometer(batch_size, 50), 
        epoch_end_callback=mx.callback.do_checkpoint(chk_prefix, 10)   ## save your model after each 10 epochs
        )

################################################################################
#                              END OF YOUR CODE                                #
################################################################################